In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/accuracy-qwen-lola/final_test.csv
/kaggle/input/accuracy-qwen-lola/final_test_predictions.csv
/kaggle/input/accuracy-qwen-lola/predicted_best_headlines.csv


In [3]:
import pandas as pd
import numpy as np

# --- Load Data ---
# 1. Ground Truth (Actual CTRs and Winner Flags)
df_ground_truth = pd.read_csv("/kaggle/input/accuracy-qwen-lola/final_test.csv")

# 2. Fine-Tuned Model Predictions (Selected Option IDs)
df_finetuned = pd.read_csv("/kaggle/input/accuracy-qwen-lola/predicted_best_headlines.csv")

# 3. Embedding Model Predictions (Predicted CTRs)
df_embedding = pd.read_csv("/kaggle/input/accuracy-qwen-lola/final_test_predictions.csv")

In [4]:
# --- Helper Function: Get Ground Truth Map ---
# Creates a dictionary {test_id: best_headline_text} and {test_id: best_option_number}
def get_ground_truth_map(df):
    # Filter for the winning rows
    winners = df[df['is_best'] == 1]
    
    # Map test_id -> best_option_number
    id_to_option = winners.set_index('test_id')['option_number'].to_dict()
    
    # Map test_id -> best_headline_text
    id_to_headline = winners.set_index('test_id')['headline'].to_dict()
    
    return id_to_option, id_to_headline

gt_option_map, gt_headline_map = get_ground_truth_map(df_ground_truth)
all_test_ids = list(gt_option_map.keys())

print(f"Total Test Groups: {len(all_test_ids)}")

Total Test Groups: 3263


In [5]:
# --- Evaluation 1: Fine-Tuned Qwen Model ---
print("\n--- Evaluating Fine-Tuned Qwen Model ---")

correct_ft = 0
total_ft = 0
missing_ft = 0

for index, row in df_finetuned.iterrows():
    tid = row['test_id']
    pred_option = row['predicted_option']
    
    if tid in gt_option_map:
        total_ft += 1
        if pred_option == gt_option_map[tid]:
            correct_ft += 1
    else:
        missing_ft += 1

accuracy_ft = (correct_ft / total_ft) * 100 if total_ft > 0 else 0
print(f"Fine-Tuned Accuracy: {accuracy_ft:.2f}% ({correct_ft}/{total_ft})")
if missing_ft > 0:
    print(f"Warning: {missing_ft} test IDs were missing from ground truth.")


--- Evaluating Fine-Tuned Qwen Model ---
Fine-Tuned Accuracy: 35.70% (1165/3263)


In [6]:
# --- Evaluation 2: Embedding Model (Regression) ---
print("\n--- Evaluating Embedding Model (Regression) ---")

# We need to group by test_id (which isn't in the embedding file explicitly, 
# so we must merge or assume order. 
# LOOKING AT THE FILES: 'final_test_predictions.csv' has 'headline' but NOT 'test_id'.
# 'final_test.csv' has both. We must merge on 'headline' to recover 'test_id'.

# Merge embedding predictions with ground truth to get test_id back
df_emb_merged = pd.merge(
    df_embedding, 
    df_ground_truth[['headline', 'test_id', 'is_best']], 
    on='headline', 
    how='inner'
)

# Now find the predicted winner for each test_id based on max 'CTR_pred'
grouped_emb = df_emb_merged.groupby('test_id')

correct_emb = 0
total_emb = 0

for tid, group in grouped_emb:
    total_emb += 1
    
    # Find the row with the highest PREDICTED CTR
    predicted_winner_row = group.loc[group['CTR_pred'].idxmax()]
    
    # Check if this row is actually the best (is_best == 1)
    # OR compare headlines text if is_best isn't reliable in the merge
    if predicted_winner_row['is_best'] == 1:
        correct_emb += 1
    elif tid in gt_headline_map:
        # Fallback: Compare headline text directly
        if predicted_winner_row['headline'] == gt_headline_map[tid]:
            correct_emb += 1

accuracy_emb = (correct_emb / total_emb) * 100 if total_emb > 0 else 0
print(f"Embedding Model Accuracy: {accuracy_emb:.2f}% ({correct_emb}/{total_emb})")


--- Evaluating Embedding Model (Regression) ---
Embedding Model Accuracy: 42.79% (1383/3232)


In [7]:
# --- Comparison Summary ---
print("\n" + "="*30)
print("🏆 FINAL RESULTS 🏆")
print("="*30)
print(f"Model A (Fine-Tuned LLM): {accuracy_ft:.2f}%")
print(f"Model B (Embedding Reg):  {accuracy_emb:.2f}%")

if accuracy_ft > accuracy_emb:
    print("\n✅ The Fine-Tuned LLM performed better.")
else:
    print("\n✅ The Embedding Regression model performed better.")


🏆 FINAL RESULTS 🏆
Model A (Fine-Tuned LLM): 35.70%
Model B (Embedding Reg):  42.79%

✅ The Embedding Regression model performed better.
